# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user requests. \
Your Database is composed of an SQL database with some tables. \
Try to make sure the SQL order is simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user asks for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [19]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
first table:
{
  "tableName": "hospitals",
  "fields": [
    {
      "name": "hosp_id",
      "type": "int"
    },
    {
      "name": "name",
      "type": "varchar"
    },
    {
      "name": "insurance",
      "type": "varchar"
    },
    {
      "name": "phone",
      "type": "varchar"
    },
    {
      "name": "address",
      "type": "varchar"
    },
    {
      "name": "city",
      "type": "varchar"
    },
    {
      "name": "state",
      "type": "varchar"
    },
    {
      "name": "specialty",
      "type": "varchar"
    },
    {
      "name": "treatment_successrate",
      "type": "int"
    },
    {
      "name": "doctors",
      "type": "int"
    },       
  ]
},
second table:
{
  "tableName": "patients",
  "fields": [
    {
      "name": "hosp_id",
      "type": "int"
    },
    {
      "name": "patient_id",
      "type": "int"
    },
    {
      "name": "doctor_id",
      "type": "int"
    },
    {
      "name": "insurance",
      "type": "varchar"
    }, 
    {
      "name": "balance",
      "type": "int"
    },      
    {
      "name": "first_name",
      "type": "varchar"
    },
    {
      "name": "last_name",
      "type": "varchar"
    },
    {
      "name": "admission_date",
      "type": "date"
    },
    {
      "name": "symptoms",
      "type": "text"
    },
    {
      "name": "diagnosis_id",
      "type": "int"
    },
    {
      "name": "phone",
      "type": "varchar"
    },
    {
      "name": "email",
      "type": "varchar"
    },
    {
      "name": "address",
      "type": "varchar"
    },
    {
      "name": "treatment_success_rate",
      "type": "int"
    }
  ]
},
third table:
{
  "tableName": "doctors",
  "fields": [
    {
      "name": "hosp_id",
      "type": "int"
    },
    {
      "name": "doctor_id",
      "type": "int"
    },
    {
      "name": "salary",
      "type": "int"
    }, 
    {
      "name": "insurance",
      "type": "varchar"
    },      
    {
      "name": "first_name",
      "type": "varchar"
    },
    {
      "name": "last_name",
      "type": "varchar"
    },
    {
      "name": "phone",
      "type": "varchar"
    },
    {
      "name": "email",
      "type": "varchar"
    },
    {
      "name": "address",
      "type": "varchar"
    },
    {
      "name": "begin_date",
      "type": "date"
    },
    {
      "name": "medical_specialty",
      "type": "varchar"
    },
    {
      "name": "diagnosis_id",
      "type": "int"
    },
    {
      "name": "med_degree",
      "type": "varchar"
    },
    {
      "name": "treatment_success_rate",
      "type": "int"
    }
  ]
},
fourth table:
{
  "tableName": "diagnoses",
  "fields": [
    {
      "name": "diagnosis_id",
      "type": "int"
    },
    {
      "name": "disease_name",
      "type": "varchar"
    },
    {
      "name": "disease_type",
      "type": "varchar"
    },
    {
      "name": "treatment_cost",
      "type": "int"
    },
    {
      "name": "insurance",
      "type": "varchar"
    },
    {
      "name": "malignant",
      "type": "boolean"
    },
    {
      "name": "disease_frequency",
      "type": "varchar"
    },
    {
      "name": "symptoms", 
"""} ]



In [11]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
which doctors started working at the hospital in 2020?
"""
})

In [12]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [16]:
#new
context_user = context.copy()
print(return_CCRMSQL("""which doctor made the most money in 2021?""", context_user))

To determine which doctor made the most money in 2021, we need to consider the salary of each doctor and filter the data based on the year 2021. Assuming the "begin_date" field in the "doctors" table represents the start date of the doctor's employment, we can calculate the total salary earned by each doctor in 2021 by considering their salary and the duration they worked in that year.

Here is a sample SQL query that can be used to find the doctor who made the most money in 2021:

```sql
SELECT 
    d.doctor_id,
    d.first_name,
    d.last_name,
    SUM(d.salary) AS total_salary_earned
FROM doctors d
WHERE YEAR(d.begin_date) = 2021
GROUP BY d.doctor_id, d.first_name, d.last_name
ORDER BY total_salary_earned DESC
LIMIT 1;
```

This query will sum up the salaries of each doctor for the year 2021, group the results by doctor, and then order them in descending order of total salary earned. The doctor who made the most money in 2021 will be the first result returned by the query.


In [17]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("what is the name of the use with the highest salary?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the user with the highest salary by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the results by salary in descending order and limits the output to only the top result, which corresponds to the user with the highest salary.


In [20]:
#new
print(return_CCRMSQL("which treatment is most expensive and is not covered by insurance?", context_user))

To determine the most expensive treatment that is not covered by insurance, we need to look at the "diagnoses" table and compare the "treatment_cost" for each diagnosis with the insurance coverage information in the "patients" table.

Here is the SQL query to find the most expensive treatment that is not covered by insurance:

```sql
SELECT d.disease_name, d.treatment_cost
FROM diagnoses d
WHERE d.treatment_cost = (
    SELECT MAX(treatment_cost)
    FROM diagnoses
    WHERE diagnosis_id NOT IN (
        SELECT diagnosis_id
        FROM patients
        WHERE insurance = 'Not Covered'
    )
);
```

This query will retrieve the disease name and treatment cost of the most expensive treatment that is not covered by insurance.


In [21]:
#old
print(return_CCRMSQL("which educational level earned the most money after graduating?", old_context_user))

This is your SQL:
```sql
SELECT s.ID_usr, MAX(s.salary) AS max_salary
FROM salary s
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY st.educational_level
```

This SQL query joins the "salary" and "studies" tables on the ID_usr column. It then calculates the maximum salary earned by each educational level after graduating by grouping the results based on the educational level.


In [25]:
#new
print(return_CCRMSQL("which patients still have not paid for their treatment?", context_user))

To determine which patients still have not paid for their treatment, you can query the "patients" table where the "balance" field is greater than 0. This query will give you a list of patients who still have outstanding balances for their treatment.


In [27]:
print(return_CCRMSQL("which patients still have an outstanding balance?", context_user))

To determine which patients still have an outstanding balance, you can query the "patients" table where the "balance" field is greater than 0. Here is an SQL query that you can use:

```sql
SELECT * 
FROM patients
WHERE balance > 0;
```

This query will retrieve all the patients who still have an outstanding balance.


In [26]:
print(return_CCRMSQL("which doctors make the least money and what is their success rate in treatment?", context_user))

To find the doctors who make the least money and their success rate in treatment, you can query the "doctors" table in the database. Here is an SQL query that can help you retrieve this information:

```sql
SELECT first_name, last_name, salary, treatment_success_rate
FROM doctors
ORDER BY salary
LIMIT 1;
```

This query will select the doctor with the lowest salary and display their first name, last name, salary, and treatment success rate.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

After running several prompts, it looks like the model is good at not only creating an SQL query for all but one prompt, but also good at translating back its findings and explaining the SQL query in human language. It takes a human input, and creates a technical output with a human explanation, which is very impressive. The only prompt it failed in was "which patients still have not paid for their treatment?", in which in provided instructions for the users to create the query themself but provided no query, which is strange because the instructions for how to build the query are correct, and that means that the model was able to translate the process, using its own words and yet did not understand the task of building the query. It took the question, understood it by putting a solution into instructive language of its own, which means it had an "idea" of what was being asked, and yet did not produce the query itself. This is interesting as I cannot understand what the limitation was that it hit. When using the precise language of the table "which patients still have an outstanding balance?" for the same question, it produces the summary and query as asked. So the the limit appears to have just been the use of synonyms, but again it's strange because it converted the synonym I used for balance "have not paid" into instructions about the balance.